In [2]:
#r "Examples\ObjectCalisthenics\bin\Debug\net7.0\ObjectCalisthenics.dll"

## Rule 1: One level of indentation per method

Methoden mit mehr als eine Einrückungsebene...  
* ... sind oft zu lang.  
* ... sind schwerer zu lesen.  
* ... haben oft mehrere Zuständigkeiten.  
    * **Verstoß gegen das Single-Responsibility-Principle!**

### Beispiel

#### Problem

Ein Renderer für ein Vier-Gewinnt-Spiel auf der Konsole:

In [3]:
using ObjectCalisthenics.Rule1.Example1.Models;

public class ConnectFourRenderer
{
    public string ToString(Board board)
    {
        // Ebene 0
        var builder = new StringBuilder();

        if (board != null)
        {
            // Ebene 1
            for (var row = 0; row < board.Height; row++)
            {
                // Ebene 2
                for (var column = 0; column < board.Width; column++)
                {
                    // Ebene 3
                    string player;
                    if (board[column, row] != null)
                    {
                        // Ebene 4
                        player = board[column, row].Player.ToString();
                    }
                    else
                    {
                        player = "-";
                    }

                    builder.Append($"{player} ");
                }

                builder.AppendLine();
            }
        }

        return builder.ToString();
    }
}

In [4]:
var board = new Board(6, 7);
board[0, 6] = new Field(Player.X);
board[0, 5] = new Field(Player.O);
board[1, 6] = new Field(Player.X);
board[0, 4] = new Field(Player.O);

Console.WriteLine(new ConnectFourRenderer().ToString(board));

- - - - - - 
- - - - - - 
- - - - - - 
- - - - - - 
O - - - - - 
O - - - - - 
X X - - - - 



#### Lösung

In [5]:
using ObjectCalisthenics.Rule1.Example1.Models;

public class ConnectFourRenderer
{
    public string ToString(Board board)
    {
        if(board == null)
            return string.Empty;

        var builder = new StringBuilder();
        RenderRows(board, builder);

        return builder.ToString();
    }

    private static void RenderRows(Board board, StringBuilder builder)
    {
        for(var row = 0; row < board.Height; row++)
            RenderRow(board, builder, row);
    }

    private static void RenderRow(Board board, StringBuilder builder, int row)
    {
        for(var column = 0; column < board.Width; column++)
            builder.Append($"{GetPlayer(board[column, row])} ");

        builder.AppendLine();
    }

    private static string GetPlayer(Field field) => field?.Player.ToString() ?? ".";
}

In [6]:
var board = new Board(6, 7);
board[0, 6] = new Field(Player.X);
board[0, 5] = new Field(Player.O);
board[1, 6] = new Field(Player.X);
board[0, 4] = new Field(Player.O);

Console.WriteLine(new ConnectFourRenderer().ToString(board));

. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
O . . . . . 
O . . . . . 
X X . . . . 



## Rule 2: Don’t use the “else” keyword

* Ebenfalls ein Indiz für mehrere Zuständigkeiten.
    * **Verstoß gegen das Single-Responsibility-Principle!**
* Die Verlockung ist groß, einer `if`-Abfrage einfach ein else/if else-Block hinzuzufügen.
    * **Verstoß gegen das Open-Close-Principle!**

### Beispiel 1

#### Problem

Klasse mit mehreren Zuständigkeiten.
Die Klasse besitzt die Methoden ``DoSomething()`` und ``DoSomethingElse()``, die zwei unterschiedliche Logiken ausführen.

Der Test wird komplexer, weil man nun beide Logiken testen muss.

In [7]:
using ObjectCalisthenics.Rule2.Example1.Models;

public class MultipleResponsibilities
{
    public void DoSomething(bool somethingElse)
    {
        if(!somethingElse)
            DoSomething();
        else
            DoSomethingElse();
    }

    private void DoSomething() { Console.WriteLine("DoSomething"); }

    private void DoSomethingElse() { Console.WriteLine("DoSomethingElse"); }
}

In [8]:
var multipleResponsibilities = new MultipleResponsibilities();
multipleResponsibilities.DoSomething(true);
multipleResponsibilities.DoSomething(false);

DoSomethingElse
DoSomething


#### Lösung

Die Logiken in separate Klassen auslagern.

In [9]:
using ObjectCalisthenics.Rule2.Example1.Models;

public class MultipleResponsibilities
{
    public void DoSomething(ISomething something)
    {
        something.Do();
    }
}

public class Something : ISomething
{
    public void Do() { Console.WriteLine("DoSomething"); }
}

public class SomethingElse : ISomething
{
    public void Do() { Console.WriteLine("DoSomethingElse"); }
}

In [10]:
var multipleResponsibilities = new MultipleResponsibilities();
multipleResponsibilities.DoSomething(new Something());
multipleResponsibilities.DoSomething(new SomethingElse());

DoSomething
DoSomethingElse


### Beispiel 2

#### Problem

Schlecht lesbarer und wartbarer Code durch ``if``, ``else if`` und ``else`` Überfrachtung.

In [2]:
using ObjectCalisthenics.Rule2.Example2.Models;

public class StateMaschine
{
    public State State { get; private set; } = State.Pending;

    public void Handle(ICommand command)
    {
        if (this.State == State.Started && command is RollDiceCommand)
            command.Execute();
        else
            throw new Exception();
    }

    public void Next()
    {
        if(this.State == State.Pending)
            this.State = State.Started;
        else if(this.State == State.Started)
            this.State = State.Finished;
        else if(this.State == State.Finished)
            throw new Exception("Machine already finished!");
        else if(this.State == State.Aborted)
            throw new Exception("Machine Aborted!");
    }

    public void Abort()
    {
        if(this.State == State.Finished)
            throw new Exception("Machine already finished!");
        else if(this.State == State.Aborted)
            throw new Exception("Machine Aborted!");

        this.State = State.Aborted;
    }
}

In [3]:
var stateMaschine = new StateMaschine();
var command = new RollDiceCommand();
stateMaschine.Handle(command);

Error: System.Exception: Exception of type 'System.Exception' was thrown.
   at Submission#3.StateMaschine.Handle(ICommand command)
   at Submission#4.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [5]:
var stateMaschine = new StateMaschine();
stateMaschine.Next();
var command = new RollDiceCommand();
stateMaschine.Handle(command);

5


#### Lösung

State Pattern!

In [18]:
public interface IState
{
    void Next();
    void Abort();
    bool CanHandle(ICommand command);
}

public class StateMaschine
{
    public IState State { get; private set; }
    public void Next() => State.Next();
    public void Abort() => State.Abort();
    public void ChangeState(IState state) => State = state;

    public StateMaschine()
    {
        State = new PendingState(this);
    }

    public void Handle(ICommand command)
    {
        if (!State.CanHandle(command))
            throw new Exception();

        command.Execute();
    }
}

public abstract class State : IState
{
    protected StateMaschine StateMaschine { get; }

    public State(StateMaschine stateMaschine) => StateMaschine = stateMaschine;

    public virtual void Abort() => StateMaschine.ChangeState(new AbortedState(StateMaschine));
    public virtual bool CanHandle(ICommand command) => false;
    public abstract void Next();
}

public class PendingState : State
{
    public PendingState(StateMaschine stateMaschine) : base(stateMaschine) { }

    public override void Next() => StateMaschine.ChangeState(new StartedState(StateMaschine));
}

public class StartedState : State
{
    public StartedState(StateMaschine stateMaschine) : base(stateMaschine) { }

    public override bool CanHandle(ICommand command) => command switch
    {
        RollDiceCommand => true,
        _ => false
    };

    public override void Next() => StateMaschine.ChangeState(new FinishedState(StateMaschine));
}

public class FinishedState : State
{
    public FinishedState(StateMaschine stateMaschine) : base(stateMaschine) { }

    public override void Abort() => throw new Exception("Machine already finished!");
    public override void Next() => throw new Exception("Machine already finished!");
}

public class AbortedState : State
{
    public AbortedState(StateMaschine stateMaschine) : base(stateMaschine) { }

    public override void Abort() => throw new Exception("Machine Aborted!");
    public override void Next() => throw new Exception("Machine Aborted!");
}

In [19]:
var stateMaschine = new StateMaschine();
var command = new RollDiceCommand();
stateMaschine.Handle(command);

Error: System.Exception: Exception of type 'System.Exception' was thrown.
   at Submission#19.StateMaschine.Handle(ICommand command)
   at Submission#20.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [20]:
var stateMaschine = new StateMaschine();
stateMaschine.Next();
var command = new RollDiceCommand();
stateMaschine.Handle(command);

4


## Rule 3: Wrap all primitives and Strings

* Wertetypen und Strings folgen Businessregeln. Beispiele:
    * IBAN
    * BIC
    * ISBN
    * E-Mail-Adresse
    * Spielwürfel
    * IDs
* Also gehört die Logik auch an eine zentrale Stelle
* Verhindert auch, dass Datentypen nicht vertauscht werden können
    * **Primitive Obsession Anti-Pattern!**

### Beispiel 1

#### Problem

Ein Rechner, der den Brutto-Betrag eines Nettos-Betrags errechnet.

In [24]:
public class GrossCalculator
{
    public static decimal Gross(decimal net, decimal taxes)
        => net + net * taxes;
}

Muss die Methode jetzt mit ``Gross(123.45m, 0.19)`` oder ``Gross(123.45m, 19)`` aufgerufen werden? Sind negative Steuern erlaubt?

In [25]:
Console.WriteLine(GrossCalculator.Gross(123.45m, 0.19m));
Console.WriteLine(GrossCalculator.Gross(123.45m, 19m));

146.9055
2469.00


#### Lösung

Separate Klassen.  
Man muss zwar immer noch die Dokumenation zu Taxes lesen aber jetzt ist klarer, dass man z.B. 19 verwenden muss.

Zwischen Netto- und Brutto-Wert wird auch unterschieden und können nicht mehr vertauscht werden.

In [27]:
public class GrossCalculator
{
    public static Gross Gross(Net net, Taxes taxes)
        => new(net.Amount + net.Amount * taxes.Percentage2);
}

public class Taxes
{
    public decimal Percentage { get; }

    public decimal Percentage2 => this.Percentage / 100;

    public Taxes(decimal Percentage)
    {
        if(Percentage <= 0 || Percentage >= 100)
            throw new ArgumentOutOfRangeException(nameof(Percentage));

        this.Percentage = Percentage;
    }
}

public record Money(decimal Amount);
public record Net(decimal Amount) : Money(Amount);
public record Gross(decimal Amount) : Money(Amount);

In [28]:
Console.WriteLine(GrossCalculator.Gross(new Net(123.45m), new Taxes(19)));

Gross { Amount = 146.9055 }


### Beispiel 2

#### Problem

Eine Klasse, die einen 6-seitigen Spielwürfel repräsentiert, liefert einen ``int``-Wert zurück.

In [29]:
public class Dice
{
    private static readonly Random _random = new();

    public int Roll() => _random.Next(1, 6 + 1);
}

Jede Methode, die das Ergebnis von ``Roll()`` verarbeitet, muss nun schauen, dass der Wert zwischen 1 und 6 liegt:

In [30]:
public void Execute(int roll)
{
    if(roll < 1 || roll > 6)
        throw new ArgumentOutOfRangeException(nameof(roll));
    // ...
}

In [31]:
Execute(7);

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'roll')
   at Submission#31.Execute(Int32 roll)
   at Submission#32.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

#### Lösung

Eigenen Datentyp für das Würfelergebnis.

In [32]:
public class Dice
{
    private static readonly Random _random = new();

    public DiceResult Roll() => new(_random.Next(1, 6 + 1));
}

public class DiceResult
{
    public int Value { get; }

    public DiceResult(int value)
    {
        if(value < 1 || value > 6)
            throw new ArgumentOutOfRangeException(nameof(value));

        this.Value = value;
    }
}

In [33]:
public void Execute(DiceResult diceResult)
{
    Console.WriteLine(diceResult);
}

In [34]:
Execute(new DiceResult(7));

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'value')
   at Submission#33.DiceResult..ctor(Int32 value)
   at Submission#35.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Rule 4: First class collections

* Wie Regel 3, nur mit Collections
* Auch Collections folgen oft Businessregeln

### Beispiel

#### Problem

Eine Klasse, die einen einen Spieler eines Kartenspiels repräsentiert, besitzt eine Liste mit den Spielkarten des Spielers.
Ein Spieler darf nur maximal fünf Karten besitzen.

In [35]:
using ObjectCalisthenics.Rule4.Example1.Models;

public class Player
{
    private readonly List<Card> _hand = new();
    public IReadOnlyCollection<Card> Hand => _hand.AsReadOnly();

    public void AddCardToHand(Card card)
    {
        if(_hand.Count == 5)
            throw new Exception();

        _hand.Add(card);
    }
}

In [36]:
var player = new Player();
player.AddCardToHand(new Card(Suit.Two, Color.Clubs));
player.AddCardToHand(new Card(Suit.Four, Color.Diamonds));
player.AddCardToHand(new Card(Suit.Six, Color.Hearts));
player.AddCardToHand(new Card(Suit.Nine, Color.Clubs));
player.AddCardToHand(new Card(Suit.Joker, Color.Hearts));
Console.WriteLine(string.Join(Environment.NewLine, player.Hand));
player.AddCardToHand(new Card(Suit.King, Color.Spades));

Card { Suit = Two, Color = Clubs }
Card { Suit = Four, Color = Diamonds }
Card { Suit = Six, Color = Hearts }
Card { Suit = Nine, Color = Clubs }
Card { Suit = Joker, Color = Hearts }


Error: System.Exception: Exception of type 'System.Exception' was thrown.
   at Submission#36.Player.AddCardToHand(Card card)
   at Submission#37.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Die ``Player``-Klasse ist nun zuständig, zu schauen, dass es nicht mehr als fünf Karten werden.

#### Lösung

Eine separate Klasse, die für die Karten zuständig ist.

In [37]:
using ObjectCalisthenics.Rule4.Example1.Models;

public class Player
{
    private readonly Hand _hand = new();
    public IReadOnlyCollection<Card> Hand => _hand.Cards;

    public void AddCardToHand(Card card) => _hand.Add(card);
}

internal class Hand
{
    private readonly List<Card> _cards = new List<Card>();
    public IReadOnlyCollection<Card> Cards => _cards.AsReadOnly();

    public void Add(Card card)
    {
        if(_cards.Count == 5)
            throw new Exception();

        _cards.Add(card);
    }
}

In [38]:
var player = new Player();
player.AddCardToHand(new Card(Suit.Two, Color.Clubs));
player.AddCardToHand(new Card(Suit.Four, Color.Diamonds));
player.AddCardToHand(new Card(Suit.Six, Color.Hearts));
player.AddCardToHand(new Card(Suit.Nine, Color.Clubs));
player.AddCardToHand(new Card(Suit.Joker, Color.Hearts));
Console.WriteLine(string.Join(Environment.NewLine, player.Hand));
player.AddCardToHand(new Card(Suit.King, Color.Spades));

Card { Suit = Two, Color = Clubs }
Card { Suit = Four, Color = Diamonds }
Card { Suit = Six, Color = Hearts }
Card { Suit = Nine, Color = Clubs }
Card { Suit = Joker, Color = Hearts }


Error: System.Exception: Exception of type 'System.Exception' was thrown.
   at Submission#38.Hand.Add(Card card)
   at Submission#38.Player.AddCardToHand(Card card)
   at Submission#39.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Dadurch wird die Komplexität der ``Player``-Klasse verringert.

## Rule 5: One dot per line

* Law of demeter
    * Objekte sollen nur mit Objekten in ihrer unmittelbaren Umgebung kommunizieren.
    * Die Kopplung (= die Anzahl von Abhängigkeiten) wird verringert und die Wartbarkeit erhöht.

### Beispiel

#### Problem

Ein Auto, mit einem Motor und ein Fahrer, der ein Auto fährt.

In [ ]:
public class Engine
{
    public void Start() { Console.WriteLine("Wruum!"); }
}

public class Car
{
    public Engine Engine { get; }

    public Car(Engine engine)
    {
        Engine = engine;
    }
}

public class Driver
{
    public void Drive(Car car)
    {
        car.Engine.Start();
    }
}

In [ ]:
var engine = new Engine();
var car = new Car(engine);
var driver = new Driver();

driver.Drive(car);

Semantisch betrachtet ist dies sogar falsch, da die Logik nun so aussieht, als würde der Fahrer mit Muskelkraft den Motor starten, da der Fahrer direkt auf den Motor zugreift aber der Fahrer gibt dem Auto ja nur das Signal, dass der Motor gestartet werden soll.

#### Lösung

Das Auto muss den Motor starten.

In [ ]:
public class Engine
{
    public void Start() { Console.WriteLine("Brumm!"); }
}

public class Car
{
    private readonly Engine _engine;

    public Car(Engine engine)
    {
        _engine = engine;
    }
    public void Start() => _engine.Start();
}

public class Driver
{
    public void Drive(Car car)
    {
        car.Start();
    }
}

In [ ]:
var engine = new Engine();
var car = new Car(engine);
var driver = new Driver();

driver.Drive(car);

## Rule 6: Don’t abbreviate

Die Frage ist: Warum möchte man abkürzen?
* Weil man die Methode immer und immer wieder aufruft?
    * Klingt nach einer Code-Duplizierung. Vielleicht ist das Design noch nicht gut genug.
        * (Meine Meinung: Ist aber mit Vorsicht zu genießen. Lieber KISS und die Methode mehrmals aufrufen, anstatt komplizierte Abstrahierungen.)
* Weil der Methodenname zu lang ist?
    * Klingt danach, dass die Methode mehrere Zuständigkeiten hat. 

## Rule 7: Keep all entities small

* Damit sind Klassen ansich gemeint.
* Eine Klasse sollte nicht mehr als 50 Zeilen haben.
    * Halte ich für sehr hart. Ich tendiere da eher so bei max. 150 Zeilen.
    * Wer mehr braucht, muss dafür schon plausible Gründe nennen.
* Mehr Zeilen kann wieder ein Indiz sein, dass die Klasse mehr als eine Zuständigkeit hat.

### Beispiel

#### Problem

Ein Report-Generator.

In [ ]:
using ObjectCalisthenics.Rule7.Example1.Models;

public class Report
{
    public void Execute(IExcel excel)
    {
        excel.SetValue("A1", CalculateFigureA());
        excel.SetValue("A2", CalculateFigureB());
        excel.SetValue("A3", CalculateFigureC());

        foreach(var (cell, value) in CalculateTable())
            excel.SetValue(cell, value);
    }

    private decimal CalculateFigureA() => CalculateFigureA_1() + CalculateFigureA_2();

    private decimal CalculateFigureA_1() => 4711;

    private decimal CalculateFigureA_2() => 23;

    private decimal CalculateFigureB() => CalculateFigureB_1() + CalculateFigureB_2() / CalculateFigureB_3();

    private decimal CalculateFigureB_1() => 42;

    private decimal CalculateFigureB_2() => 3.14m;

    private decimal CalculateFigureB_3() => 69;

    private decimal CalculateFigureC() => 100;

    private IEnumerable<(string cell, decimal value)> CalculateTable()
        => new (string cell, decimal value)[]
        {
            new("A2", 10),
            new("B2", 20),
            new("C2", 30),
            new("D2", 40),
        };
}

In [ ]:
var report = new Report();
var excel = new Excel();
report.Execute(excel);

Es werden mehrere Zuständigkeiten vermischt:
* Das Tranferieren der Werte nach Excel
* Die Berechnung mehreren von Kennzahlen

Weder das Transferieren, noch die Kennzahlberechnung kann gezielt und isoliert getestet werden.

#### Lösung

Das Aufzeilen in ihre Zuständigkeiten

In [ ]:
using ObjectCalisthenics.Rule7.Example1.Models;

public class Report
{
    public void Execute(IExcel excel)
    {
        excel.SetValue("A1", new FigureA().Calculate());
        excel.SetValue("A2", new FigureB().Calculate());
        excel.SetValue("A3", new FigureC().Calculate());

        var row = 10;
        foreach(var (country, value) in new Table().Calculate())
        {
            excel.SetValue($"A{row}", country);
            excel.SetValue($"B{row}", value);
            row++;
        }
    }
}

public interface IFigure<T>
{
    T Calculate();
}

public interface ITable<T> : IFigure<IEnumerable<T>> { }

public class FigureA : IFigure<decimal>
{
    public decimal Calculate() => CalculateFigureA_1() + CalculateFigureA_2();

    private decimal CalculateFigureA_1() => 4711;

    private decimal CalculateFigureA_2() => 23;
}

public class FigureB : IFigure<decimal>
{
    public decimal Calculate() => CalculateFigureB_1() + CalculateFigureB_2() / CalculateFigureB_3();

    private decimal CalculateFigureB_1() => 42;

    private decimal CalculateFigureB_2() => 3.14m;

    private decimal CalculateFigureB_3() => 69;
}

public class FigureC : IFigure<decimal>
{
    public decimal Calculate() => 100;
}

public class Table : ITable<(string country, decimal value)>
{
    public IEnumerable<(string country, decimal value)> Calculate()
        => new (string country, decimal value)[]
        {
            new("Deutschland", 10),
            new("Dänemark", 20),
            new("Frankreich", 30),
            new("Italien", 40),
        };
}

In [ ]:
var figureA = new FigureA();
var figureB = new FigureB();
var figureC = new FigureC();
var table = new Table();
Console.WriteLine(figureA.Calculate());
Console.WriteLine(figureB.Calculate());
Console.WriteLine(figureC.Calculate());
table.Calculate().ToList().ForEach(x => Console.WriteLine($"{x.country} => {x.value}"));

var report = new Report();
var excel = new Excel();
report.Execute(excel);

## Rule 8: Do not use classes with more than two instance variables

* Die Regel ist absichtlich provokativ gehalten. 
    * Die Begründung ist: "Warum nicht?"
* Sie soll einen dazu anregen, welche Daten wie zusammengefasst werden können.

### Beispiel

#### Problem

Eine ``Person``-Klasse, die sehr viele Eigenschaften beinhaltet.

In [ ]:
public record Customer(Guid Id, string FirstName, string LastName, string Street, string StreetNo, string City, string PostalCode);

#### Lösung

Wenn man es aber genau betrachtet, lasssen sich viele Dinge clustern.

In [ ]:
public record class Customer(Guid Id, Contact Contact);
public record Contact(Name Name, Address Address);
public record Name(string FirstName, string LastName);
public record Street(string Name, string No);
public record Address(Street street, string City, string PostalCode); // Hier weiche ich schon ab. Mir fällt da  nichts mehr besseres ein.

Das geht ebenfalls in die Richtung von Regel 3 (Wrap all primitives and Strings).

## Rule 9: No getters/setters/properties

* Getter sind zwar schon erlaubt, um Dinge abzufragen, sie sollen aber kein Bestandteil der Logik innerhalb der Klasse sein.
* Gemeint ist das "Tell, don't ask"-Principle.

### Beispiel

#### Problem

In einem Spiel soll mit einer Methode der Score eines Spielers um 10 Punkte erhöht werden.

In [ ]:
public record Score(int Points);

public class Player
{
    public Score Score {get; private set; } = new Score(0);

    public void UpdateScore()
    {
        Score = Score with { Points = this.Score.Points + 10 };
    }
}

In [ ]:
var player = new Player();
player.UpdateScore();
Console.WriteLine(player.Score);

Die ``Player``-Klasse ist für die Logik zuständig. Sie fragt zuerst den Score ab und addiert selber 10 Punkte drauf.  
Dies bringt wieder mehr Komplexität in die ``Player``-Klasse.

#### Lösung

Logik in die in das ``Score``-Record verschieben.

In [ ]:
public record Score(int Points)
{
    public Score Update() => this with { Points = Points + 10 };
}

public class Player
{
    public Score Score {get; private set; } = new Score(0);

    public void UpdateScore() => Score = Score.Update();
}

In [ ]:
var player = new Player();
player.UpdateScore();
Console.WriteLine(player.Score);